# Naive Bayes Spam Detection

The purpose of this notebook is to parse and classify a collection of emails as either `non-spam` or `spam` using a Naive Bayes classifier.

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from typing import List

### 1. Read emails from file and parse into words 

In [102]:
import re

# read emails from file and parse into words
train_dir = './train-mails/'
test_dir = './test-mails/'
fpaths = [os.path.join(train_dir,fname) for fname in os.listdir(train_dir)]

norm_words = []
spam_words = []

regex = re.compile('[A-z]{2,}')

print('Reading files...', end="")
for path in fpaths:
    with open(path) as file:
        for i,line in enumerate(file):
            if i == 2:
                new_words = regex.findall(line)
                
                if os.path.split(path)[1][:3] == 'spm':
                    spam_words.extend(new_words)
                else:
                    norm_words.extend(new_words)
            
            

print('complete')

Reading files...complete


### Create design matrix

1. Generate list of 1,500 most common words
2. Count words frequencies in each email
3. Assign response variable (spam/non-spam)

In [251]:
def parse_word_counts(fdir, words):
    
    # initialize design matrix
    num_words = len(words)
    X = np.zeros((len(fpaths),num_words))
    y = np.zeros(len(fpaths))

    print('Reading files...', end="")
    for i,path in enumerate(fpaths):
        with open(path) as file:
            for j,line in enumerate(file):
                if j == 2:
                    file_words = regex.findall(line)
                    n = len(file_words)

                    # record response var
                    y[i] = os.path.split(path)[1][:3] == 'spm'

                    # count words
                    for k,word in enumerate(words):
                        X[i,k] = file_words.count(word)/n
                        
    print('complete')
    
    return X, y

In [252]:
from collections import Counter

# count word frequencies and restrict to the most common
num_words = 3000
word_counts = Counter(norm_words + spam_words)
word_counts = word_counts.most_common(num_words)
common_words = [word[0] for word in word_counts]

# initialize design matrices
X_train, y_train = parse_word_counts(train_dir, common_words)
X_test, y_test = parse_word_counts(test_dir, common_words)


Reading files...complete
Reading files...complete


In [151]:
X_train.sum(axis=1).min()

0.27011494252873564

### Train model

Train the model and test the classification accuracy on the 3000 most common words

In [303]:
from sklearn.metrics import accuracy_score, precision_score

# train classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train,y_train)

# test classifier accuracy
y_pred = nb_classifier.predict(X_test)

print('Accuracy = %0.2f, Precision = %0.2f' % \
      (accuracy_score(y_test,y_pred),precision_score(y_test,y_pred)))

Accuracy = 0.86, Precision = 0.93


### Feature selection

The accuracy is quite high, but the data and model are slow to preprocess and train. Let's try just using the 20 most commonly appearing words.

In [306]:
# retrain with only the 20 most informative words
X_train, y_train = parse_word_counts(train_dir, common_words[:20])
X_test, y_test = parse_word_counts(test_dir, common_words[:20])

# train classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train,y_train)

# test classifier accuracy
y_pred = nb_classifier.predict(X_test)

print('Accuracy = %0.2f, Precision = %0.2f' % \
      (accuracy_score(y_test,y_pred),precision_score(y_test,y_pred)))

[print(word) for word in common_words[:20]];

Reading files...complete
Reading files...complete
Accuracy = 0.86, Precision = 0.93
mail
order
address
report
language
send
email
program
our
list
one
name
money
receive
free
work
information
business
please
day


An accuracy of 0.86 is not bad for just picking the most commonly used words, but I suspect we can do better by choosing the 20 most differentially distributed words between spam and normal mail. We can use the KL divergence to get a score of how different the frequencies are between spam and normal mail.

In [305]:
from scipy.stats import entropy

# find the most differentially distributed counts
norm_x = X_train[y_train>0,:]
spam_x = X_train[y_train==0,:]

bins = np.linspace(0,.1,10)
kl_div = np.zeros(num_words)

for i in range(norm_x.shape[1]):
    p_x = np.histogram(norm_x[:,i], bins=bins, density=True)[0]
    q_x = np.histogram(spam_x[:,i], bins=bins, density=True)[0]
    kl_div[i] = entropy(p_x,q_x)

One potential pitfall of this approach is that the KL divergence will go to `Inf` if our words appear in one list and not the other. For that reason, I'll restrict the resulting scores for finite KL divergence. Doing so gives us the 20 most informative words:

In [304]:
# list 20 most informative words
sort_idx = np.argsort(kl_div)[::-1]
sort_idx = sort_idx[~np.isinf(kl_div[sort_idx])]
[print(common_words[i]) for i in sort_idx[:20]];

language
order
linguistic
university
edu
english
de
subject
linguist
post
linguistics
reference
department
research
between
anyone
game
grammar
theory
word


In [309]:
# retrain with only the 20 most informative words
most_informative_words = [common_words[i] for i in sort_idx[:20]]
X_train, y_train = parse_word_counts(train_dir, most_informative_words)
X_test, y_test = parse_word_counts(test_dir, most_informative_words)

# train classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train,y_train)

# test classifier accuracy
y_pred = nb_classifier.predict(X_test)

print('Accuracy = %0.2f, Precision = %0.2f' % \
      (accuracy_score(y_test,y_pred),precision_score(y_test,y_pred)))

Reading files...complete
Reading files...complete
Accuracy = 0.96, Precision = 0.94


Not bad. Restricting to the 20 most differentially distributed words results in a 10 pt. improvement in the accurancy